In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk

events = pd.read_csv('events_up_to_01062018.csv')
training = pd.read_csv('labels_training_set.csv')

/home/martin/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, r2_score, classification_report, mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [3]:
events.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
training.head()

,person,label
0,0566e9c1,0
1,6ec7ee77,0
2,abe7a2fb,0
3,34728364,0
4,87ed62de,0


In [5]:
eventsWithLabel = events.merge(training, how='left', left_on='person', right_on='person')
eventsWithLabel.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,label
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
eventsWithLabel['timestamp'] = pd.to_datetime(eventsWithLabel['timestamp'])

In [7]:
groupbyPersonas = eventsWithLabel.loc[:,['person', 'event']].groupby('person')
dicc = {}
for clave, data in groupbyPersonas:
    lista = []
    lista = data.drop_duplicates(subset = 'event', keep = 'first')['event'].tolist()
    dicc[clave] = lista

In [8]:
#Obtengo una lista de todos los eventos que tiene esa persona
eventsWithLabel['eventos'] = eventsWithLabel['person'].map(lambda x: dicc[x])
#Obtengo los dias de la semana
eventsWithLabel['dayweek'] = eventsWithLabel.timestamp.dt.dayofweek
#Obtengo la fecha
eventsWithLabel['fecha'] = eventsWithLabel.timestamp.dt.date
#Separo la marca
eventsWithLabel['marca'] = eventsWithLabel['model'].map(lambda x: str(x).split()[0])

In [9]:
#cantidad de personas que ingresaron 27624.
groupbyCantEventos = eventsWithLabel.groupby('person', as_index = False)['event'].agg({'cantEventos': 'count'})
groupbyCantEventos.sort_values(by = 'cantEventos', ascending = False)
eventsWithLabel = pd.merge(eventsWithLabel, groupbyCantEventos, on = 'person')

In [10]:
#Filtro a la gente que no tiene el evento visited_site
personasConVisitas = eventsWithLabel.loc[eventsWithLabel['event'] == 'visited site']\
                             .drop_duplicates(subset = 'person', keep = 'first')['person'].tolist()
dataNotVisit = eventsWithLabel.loc[eventsWithLabel['person'].isin(personasConVisitas)]

In [11]:
eventsWithLabel2 = dataNotVisit.loc[dataNotVisit['event'] == 'visited site'].groupby(['person', 'fecha'], as_index = False).agg({'city': 'first', 'region': 'first', 'country': 'first', 'device_type': 'first', 'operating_system_version': 'first', 'channel': 'first', 'new_vs_returning': 'first'})

In [12]:
eventsWithLabel3 = pd.merge(dataNotVisit, eventsWithLabel2, on = ['person', 'fecha'])
eventsWithLabel3.drop(['region_x', 'device_type_x', 'operating_system_version_x', 'country_x', 'city_x', 'channel_x', 'new_vs_returning_x'], axis = 1, inplace = True)
eventsWithLabel3.rename(columns = {'region_y': 'region', 'device_type_y': 'device_type', 'operating_system_version_y': 'SO', 'city_y': 'city', 'country_y': 'country', 'channel_y': 'channel', 'new_vs_returning_y': 'new_vs_returning'}, inplace =True)

In [13]:
def obtenerSistema(so):
    sistema = so.split()
    if len(sistema) > 1:
        otro = sistema[1].split('.')[0]
        return sistema[0] + ' ' + otro
    else:
        return sistema[0]

In [14]:
eventsWithLabel3['SO'] = eventsWithLabel3['SO'].map(obtenerSistema)

In [15]:
eventosUnicos = eventsWithLabel3.drop_duplicates(subset="event", keep="first").reset_index()["event"]

In [16]:
label_encoder = LabelEncoder()
label_encoder.fit(eventosUnicos)
integer_encoded = label_encoder.transform(eventsWithLabel3["event"])

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)


/home/martin/miniconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [17]:
for i, event in enumerate(eventosUnicos):
    eventsWithLabel3[event] = onehot_encoded[:, label_encoder.transform(eventosUnicos)[i]]

In [19]:
eventsWithLabel3.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,viewed product,search engine hit,checkout,searched products,generic listing,visited site,ad campaign hit,brand listing,lead,conversion
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2018-05-18 00:30:30,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2018-05-18 00:07:23,search engine hit,4886f805,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2018-05-18 00:11:56,checkout,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2018-05-18 00:11:35,viewed product,4886f805,NaN,9287.0,Samsung Galaxy J7 Prime,Muito Bom,32GB,Dourado,NaN,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
eventsWithLabel3['condition'].value_counts()

Bom                   541428
Excelente             374870
Muito Bom             354901
Bom - Sem Touch ID     31391
Novo                    3397
Name: condition, dtype: int64

In [21]:
def conditionType(x):
    conditions = {
        "Bom - Sem Touch ID" : 1,
        "Bom" : 2,
        "Muito Bom" : 3,
        "Excelente" : 4,
        "Novo" : 5
    }
    return conditions[x] if x in conditions else 0

In [22]:
eventsWithLabel3['condition'] = eventsWithLabel3['condition'].apply(conditionType)

In [23]:
top20Colors = eventsWithLabel3['color'].value_counts().nlargest(20).index
top20Colors

Index(['Preto', 'Dourado', 'Cinza espacial', 'Branco', 'Prateado', 'Ouro Rosa',
       'Rosa', 'Prata', 'Preto Matte', 'Azul', 'Platinum', 'Preto Brilhante',
       'Ametista', 'Ouro', 'Cinza', 'Vermelho', 'Bambu', 'Verde',
       'Black Piano', 'Preto Vermelho'],
      dtype='object')

In [24]:
def colorGrouping(x):
    return x if x in top20Colors else 'Otro'

In [25]:
eventsWithLabel3['color'] = eventsWithLabel3['color'].apply(colorGrouping)

In [26]:
eventsWithLabel3.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,viewed product,search engine hit,checkout,searched products,generic listing,visited site,ad campaign hit,brand listing,lead,conversion
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,4,32GB,Dourado,NaN,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2018-05-18 00:30:30,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,4,32GB,Dourado,NaN,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2018-05-18 00:07:23,search engine hit,4886f805,NaN,NaN,NaN,0,NaN,Otro,NaN,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2018-05-18 00:11:56,checkout,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,4,32GB,Dourado,NaN,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2018-05-18 00:11:35,viewed product,4886f805,NaN,9287.0,Samsung Galaxy J7 Prime,3,32GB,Dourado,NaN,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
eventsWithLabel3['color'] = eventsWithLabel3['color'].fillna(value='idk')
coloresUnicos = eventsWithLabel3.drop_duplicates(subset='color', keep='first').reset_index()
coloresUnicos['color']

0              Dourado
1                  idk
2       Cinza espacial
3             Prateado
4               Branco
5            Ouro Rosa
6      Preto Brilhante
7                Preto
8          Preto Matte
9                 Rosa
10     Olympic Edition
11            Vermelho
12               Bambu
13      Preto Vermelho
14                Ouro
15            Platinum
16             Titânio
17     Branco Vermelho
18               Prata
19    Branco Framboesa
20         Azul Escuro
21                Azul
22          Verde Água
23        Branco Bambu
24                Iuna
25          Preto Azul
26        Couro Marrom
27              Indigo
28         Branco Azul
29       Couro Vintage
            ...       
34         Azul Safira
35        Azul Topázio
36                Roxo
37               Rouge
38               Coral
39            Ametista
40        Preto Tabaco
41         Black Piano
42            Cabernet
43               Verde
44             Amarelo
45    Branco Azul Navy
46         

In [33]:
le.fit(coloresUnicos['color'])
eventsWithLabel3['color'] = le.fit_transform(eventsWithLabel3['color'])

In [21]:
eventsWithLabel3['storage'].value_counts()

16GB     436689
32GB     425751
64GB     226577
128GB     98510
8GB       94320
256GB     17501
4GB        5318
512MB      1321
Name: storage, dtype: int64

In [73]:
def storageType(x):
    storages = {
        "512MB" : 1,
        "4GB" : 2,
        "8GB" : 3,
        "16GB" : 4,
        "32GB" : 5,
        "64GB" : 6,
        "128GB" : 7,
        "256GB" : 8
    }
    return storages[x] if x in storages else 0

In [74]:
eventsWithLabel3['storage'] = eventsWithLabel3['storage'].apply(storageType)

In [22]:
eventsWithLabel3['model'].value_counts()

iPhone 6                                     106118
iPhone 5s                                    100448
iPhone 6S                                     96485
iPhone 7                                      58598
Samsung Galaxy S7 Edge                        49915
iPhone 7 Plus                                 44627
Samsung Galaxy S7                             39279
iPhone 6S Plus                                37653
iPhone SE                                     36671
Samsung Galaxy S8                             32513
iPhone 6 Plus                                 32000
Samsung Galaxy S6 Edge                        30555
Samsung Galaxy J5                             29121
iPhone 5c                                     27763
Samsung Galaxy S6 Flat                        27485
Motorola Moto G4 Plus                         24888
Samsung Galaxy J7 Prime                       24808
Samsung Galaxy S8 Plus                        20804
iPhone 5                                      18388
Samsung Gala

In [36]:
eventsWithLabel3['model'] = eventsWithLabel3['model'].fillna(value='idk')
modelosUnicos = eventsWithLabel3.drop_duplicates(subset='model', keep='first').reset_index()
modelosUnicos['model']

0                        Samsung Galaxy J7 Prime
1                                            idk
2                                      iPhone 5s
3                                      iPhone SE
4                                       iPhone 5
5                                      iPhone 6S
6                                       iPhone 6
7                          iPad Air 2 Wi-Fi + 4G
8                              iPad Mini 4 Wi-Fi
9                            iPad Air Wi-Fi + 4G
10                             iPad Mini 3 Wi-Fi
11                                iPhone 6S Plus
12                                      iPhone X
13                                 iPhone 7 Plus
14                                      iPhone 7
15                        Samsung Galaxy S7 Edge
16                                 iPhone 6 Plus
17                     Motorola Moto G4 Play DTV
18                         Motorola Moto G4 Plus
19                         Motorola Moto G5 Plus
20                  

In [37]:
le.fit(modelosUnicos['model'])
eventsWithLabel3['model'] = le.fit_transform(eventsWithLabel3['model'])

In [38]:
eventsWithLabel3['search_term'] = eventsWithLabel3['search_term'].fillna(value='idk')
searchTermUnicos = eventsWithLabel3.drop_duplicates(subset='search_term', keep='first').reset_index()
searchTermUnicos['search_term']

0                                                idk
1                                                 J7
2                                           iphone 6
3                                          iphone 6s
4                                               ipad
5                                             iphone
6                                            moto e4
7                                              lg q6
8                                          iPhone se
9                                          iPhone SE
10                                            6splus
11                                    iphone 6s 32gb
12                                           zenfone
13                                    asus zenfone 4
14                                                e4
15                                              asus
16                                                g5
17                                                j5
18                                            

In [39]:
le.fit(searchTermUnicos['search_term'])
eventsWithLabel3['search_term'] = le.fit_transform(eventsWithLabel3['search_term'])

In [76]:
eventsWithLabel3['campaign_source'].value_counts()

google              122328
criteo               24721
rtbhouse             23357
zanox                 6849
emblue                3588
afilio                1888
manifest              1363
FacebookAds           1295
voxus                  966
bing                   837
rakuten                821
datacrush              417
indexa                 313
buscape                271
Facebook               202
afiliado                92
blog                    88
mercadopago             74
yotpo                   54
FacebookSocial          12
onsite                  12
MARKETING SOCIAL         7
gizmodo                  4
Name: campaign_source, dtype: int64

In [40]:
eventsWithLabel3['campaign_source'] = eventsWithLabel3['campaign_source'].fillna(value='idk')
campaingUnicos = eventsWithLabel3.drop_duplicates(subset='campaign_source', keep='first').reset_index()
campaingUnicos['campaign_source']

0                  idk
1               criteo
2               google
3             rtbhouse
4                zanox
5               emblue
6               afilio
7             manifest
8          FacebookAds
9                voxus
10             rakuten
11               yotpo
12              indexa
13           datacrush
14                bing
15            Facebook
16      FacebookSocial
17                blog
18             buscape
19         mercadopago
20            afiliado
21              onsite
22    MARKETING SOCIAL
23             gizmodo
Name: campaign_source, dtype: object

In [41]:
le.fit(campaingUnicos['campaign_source'])
eventsWithLabel3['campaign_source'] = le.fit_transform(eventsWithLabel3['campaign_source'])

In [77]:
eventsWithLabel3['search_engine'].value_counts()

Google    104475
Bing         676
Yahoo        518
Ask           16
Name: search_engine, dtype: int64

In [78]:
eventsWithLabel3['search_engine'] = eventsWithLabel3['search_engine'].fillna(value='idk')
searchEngineUnicos = eventsWithLabel3.drop_duplicates(subset='search_engine', keep='first').reset_index()["search_engine"]

In [ ]:
label_encoder.fit(searchEngineUnicos)
integer_encoded = label_encoder.transform(eventsWithLabel3["search_engine"])

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

In [ ]:
for i, event in enumerate(searchEngineUnicos):
    eventsWithLabel3[event] = onehot_encoded[:, label_encoder.transform(searchEngineUnicos)[i]]

In [46]:
eventsWithLabel3['browser_version'] = eventsWithLabel3['browser_version'].fillna(value='idk')
browserUnicos = eventsWithLabel3.drop_duplicates(subset='browser_version', keep='first').reset_index()
browserUnicos['browser_version']

0                                  idk
1                   Chrome Mobile 64.0
2                   Chrome Mobile 66.0
3                   Chrome Mobile 65.0
4                   Chrome Mobile 61.0
5                          Chrome 66.0
6                   Chrome Mobile 63.0
7                          Chrome 67.0
8                          Chrome 65.0
9                          Safari 11.1
10                         Android 4.1
11                         Chrome 64.0
12                    Mobile Safari 10
13                  Chrome Mobile 44.0
14                          Opera 52.0
15                          Opera 53.0
16                          Opera 51.0
17                         Chrome 63.0
18                        IE Mobile 11
19                Samsung Internet 6.4
20                Samsung Internet 3.3
21                    Mobile Safari 11
22                  Chrome Mobile 40.0
23                         Chrome 22.0
24                       Edge 16.16299
25              Chrome Mo

In [47]:
le.fit(browserUnicos['browser_version'])
eventsWithLabel3['browser_version'] = le.fit_transform(eventsWithLabel3['browser_version'])

In [79]:
marcaUnicos = eventsWithLabel3.drop_duplicates(subset='marca', keep='first').reset_index()["marca"]

In [ ]:
label_encoder.fit(marcaUnicos)
integer_encoded = label_encoder.transform(eventsWithLabel3["marca"])

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

In [ ]:
for i, event in enumerate(marcaUnicos):
    eventsWithLabel3[event] = onehot_encoded[:, label_encoder.transform(marcaUnicos)[i]]

In [23]:
eventsWithLabel3['city'].value_counts()

Unknown                    413874
São Paulo                  311035
Rio de Janeiro              94380
Belo Horizonte              69611
Salvador                    65214
Brasília                    40669
Curitiba                    37352
Osasco                      32585
Fortaleza                   27633
Recife                      25259
Guarulhos                   24912
Goiânia                     21025
Porto Alegre                19759
São Luís                    19555
São Bernardo do Campo       17182
Belém                       15266
Campinas                    15086
Natal                       12931
Santo André                 12059
Manaus                      11434
Barueri                     11232
Nova Iguaçu                 10833
Teresina                    10151
Vila Velha                  10098
Serra                        9803
Sorocaba                     8834
Sao Goncalo                  8823
Petrolina                    8390
Niterói                      8109
Vitória       

In [50]:
eventsWithLabel3['city'] = eventsWithLabel3['city'].fillna(value='idk')
ciudadUnicos = eventsWithLabel3.drop_duplicates(subset='city', keep='first').reset_index()
ciudadUnicos['city']

0                  Cantagalo
1                  São Paulo
2             Rio de Janeiro
3                    Niterói
4                Nova Iguaçu
5            Duque de Caxias
6               Praia Grande
7                  Juquitiba
8                     Osasco
9                     Santos
10                   Unknown
11                Vespasiano
12               Santa Luzia
13            Belo Horizonte
14                    Lavras
15                 Barbacena
16                   Juatuba
17                 Fortaleza
18        Ribeirão das Neves
19                 Guarulhos
20                    Manaus
21          Feira de Santana
22                   Joaçaba
23                Acailandia
24                Dom Eliseu
25                     Belém
26                  Itaborai
27           Itaquaquecetuba
28                     Macaé
29         Limoeiro do Norte
                ...         
2145              Laurentino
2146                    Mata
2147                 Caseros
2148          

In [51]:
le.fit(ciudadUnicos['city'])
eventsWithLabel3['city'] = le.fit_transform(eventsWithLabel3['city'])

In [24]:
eventsWithLabel3['region'].value_counts()

Sao Paulo                       656167
Unknown                         385171
Minas Gerais                    214847
Rio de Janeiro                  190169
Bahia                           146897
Pernambuco                       84829
Parana                           71952
Ceara                            69104
Rio Grande do Sul                58773
Federal District                 45338
Maranhao                         45101
Espirito Santo                   44737
Goias                            39476
Para                             36720
Santa Catarina                   36596
Rio Grande do Norte              29934
Paraíba                          23478
Piaui                            23136
Alagoas                          19547
Sergipe                          18940
Amazonas                         15877
Mato Grosso                      12402
Mato Grosso do Sul               11421
Tocantins                        11146
Rondonia                          8070
Acre                     

In [52]:
eventsWithLabel3['region'] = eventsWithLabel3['region'].fillna(value='idk')
regionUnicos = eventsWithLabel3.drop_duplicates(subset='region', keep='first').reset_index()
regionUnicos['region']

0                                       Rio de Janeiro
1                                            Sao Paulo
2                                              Unknown
3                                         Minas Gerais
4                                                Ceara
5                                             Amazonas
6                                                Bahia
7                                       Santa Catarina
8                                             Maranhao
9                                                 Para
10                                 Rio Grande do Norte
11                                   Rio Grande do Sul
12                                               Goias
13                                      Espirito Santo
14                                    Federal District
15                                              Parana
16                                           Tocantins
17                                  Mato Grosso do Sul
18        

In [53]:
le.fit(regionUnicos['region'])
eventsWithLabel3['region'] = le.fit_transform(eventsWithLabel3['region'])

In [25]:
eventsWithLabel3['country'].value_counts()

Brazil                   2248653
Unknown                    59399
United States               7062
Argentina                    935
Canada                       785
France                       241
Netherlands                  241
Uruguay                      226
Italy                        218
Paraguay                     212
Guadeloupe                   143
Germany                      114
Ukraine                       89
United Kingdom                85
Mexico                        74
Singapore                     65
Bulgaria                      61
Bolivia                       61
Costa Rica                    56
Spain                         54
Portugal                      48
Mozambique                    43
South Africa                  42
Israel                        37
Ireland                       35
Bangladesh                    34
Vietnam                       31
Belgium                       28
Austria                       26
Morocco                       24
Cape Verde

In [54]:
eventsWithLabel3['country'] = eventsWithLabel3['country'].fillna(value='idk')
paisUnicos = eventsWithLabel3.drop_duplicates(subset='country', keep='first').reset_index()
paisUnicos['country']

0                    Brazil
1                   Unknown
2             United States
3                    Canada
4                    France
5                     Spain
6                     Italy
7                   Germany
8                 Argentina
9                    Mexico
10                 Paraguay
11             South Africa
12              Netherlands
13                  Belgium
14       Dominican Republic
15                  Jamaica
16                   Russia
17                  Ukraine
18                  Uruguay
19                 Portugal
20                  Vietnam
21                  Georgia
22               Cape Verde
23                     Peru
24               Costa Rica
25               Mozambique
26                  Algeria
27               Bangladesh
28               Guadeloupe
29                   Israel
30                   Sweden
31           United Kingdom
32                  Ireland
33                    India
34                Singapore
35                  

In [55]:
le.fit(paisUnicos['country'])
eventsWithLabel3['country'] = le.fit_transform(eventsWithLabel3['country'])

In [80]:
deviceUnicos = eventsWithLabel3.drop_duplicates(subset='device_type', keep='first').reset_index()["device_type"]

In [ ]:
label_encoder.fit(deviceUnicos)
integer_encoded = label_encoder.transform(eventsWithLabel3["device_type"])

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

In [ ]:
for i, event in enumerate(deviceUnicos):
    eventsWithLabel3[event] = onehot_encoded[:, label_encoder.transform(deviceUnicos)[i]]

In [26]:
eventsWithLabel3['SO'].value_counts()

Windows 7        611643
Windows 10       534213
Android 6        348484
Android 7        301217
Android 5        175709
Windows 8        112590
Android 4         93172
iOS 11            36629
iOS 10            20244
Android 8         19749
Windows XP        16184
Mac OS            11398
Windows Phone      8157
Linux              8020
iOS 9              7318
Ubuntu             3453
Windows Vista      2590
Chrome OS          2553
iOS 7              1571
Other              1255
Android             955
BlackBerry OS       516
iOS 8               504
Tizen 3             308
Android 2           306
iOS 3               106
Windows RT          102
Fedora               92
Tizen 2              79
iOS 5                50
Android 10           46
Android 3            45
iOS 6                15
FreeBSD               8
Symbian OS            4
Name: SO, dtype: int64

In [58]:
eventsWithLabel3['SO'] = eventsWithLabel3['SO'].fillna(value='idk')
SOUnicos = eventsWithLabel3.drop_duplicates(subset='SO', keep='first').reset_index()
SOUnicos['SO']

0         Android 7
1         Android 5
2         Android 6
3         Windows 8
4         Windows 7
5        Windows 10
6         Android 4
7            Mac OS
8            iOS 10
9     Windows Phone
10           iOS 11
11            Linux
12            iOS 7
13            iOS 9
14        Android 8
15       Windows XP
16        Chrome OS
17           Ubuntu
18          Android
19            Other
20            iOS 8
21    Windows Vista
22          Tizen 3
23    BlackBerry OS
24          FreeBSD
25        Android 2
26        Android 3
27           Fedora
28            iOS 3
29          Tizen 2
30       Windows RT
31            iOS 6
32            iOS 5
33       Symbian OS
34       Android 10
Name: SO, dtype: object

In [59]:
le.fit(SOUnicos['SO'])
eventsWithLabel3['SO'] = le.fit_transform(eventsWithLabel3['SO'])

In [27]:
eventsWithLabel3['channel'].value_counts()

Paid        1152007
Direct       571308
Organic      398602
Referral     177169
Social        16032
Email          4117
Unknown          50
Name: channel, dtype: int64

In [81]:
channelUnicos = eventsWithLabel3.drop_duplicates(subset='channel', keep='first').reset_index()["channel"]

In [ ]:
label_encoder.fit(channelUnicos)
integer_encoded = label_encoder.transform(eventsWithLabel3["channel"])

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

In [ ]:
for i, event in enumerate(channelUnicos):
    eventsWithLabel3[event] = onehot_encoded[:, label_encoder.transform(channelUnicos)[i]]

In [62]:
eventsWithLabel3['person'] = eventsWithLabel3['person'].fillna(value='idk')
personUnicos = eventsWithLabel3.drop_duplicates(subset='person', keep='first').reset_index()
personUnicos['person']

0        4886f805
1        ad93850f
2        0297fc1e
3        2d681dd8
4        cccea85e
5        4c8a8b93
6        1b9f7cf6
7        29ebb414
8        de8fe91b
9        45baf068
10       99abca5a
11       3dc1950f
12       8ea4c165
13       64f45e8d
14       d8cfe234
15       ade6b3bf
16       d73ae9ef
17       d6bc64df
18       ec9c4059
19       b2cde41a
20       15ea8012
21       43790d8f
22       2a724d87
23       429c8732
24       686c49c9
25       4e03ed9a
26       a08d42ea
27       16fad36a
28       be32e904
29       6e6770c3
           ...   
38212    24f53ba2
38213    b8ae9999
38214    4e92e3f5
38215    39cf8fa0
38216    f85da107
38217    fd2c209b
38218    f4b2172d
38219    3adf7ca4
38220    c9552fad
38221    ef4e52ab
38222    100f33c8
38223    85e0f62a
38224    0afaecc8
38225    2c209f87
38226    4ddb8c19
38227    9979305a
38228    25bd8078
38229    87d306fc
38230    a2b1e355
38231    fb88a7ea
38232    1315ce31
38233    af3374ad
38234    023581f9
38235    9707cd0e
38236    8

In [63]:
le.fit(personUnicos['person'])
eventsWithLabel3['person'] = le.fit_transform(eventsWithLabel3['person'])

In [64]:
eventsWithLabel3['fecha'] = eventsWithLabel3['fecha'].fillna(value='idk')
fechaUnicos = eventsWithLabel3.drop_duplicates(subset='fecha', keep='first').reset_index()
fechaUnicos['fecha']

0      2018-05-18
1      2018-05-16
2      2018-05-22
3      2018-05-14
4      2018-05-19
5      2018-05-03
6      2018-05-28
7      2018-05-21
8      2018-05-23
9      2018-05-26
10     2018-05-24
11     2018-05-10
12     2018-05-25
13     2018-05-17
14     2018-03-12
15     2018-03-15
16     2018-04-20
17     2018-03-20
18     2018-04-23
19     2018-04-24
20     2018-04-02
21     2018-05-02
22     2018-04-17
23     2018-03-22
24     2018-04-06
25     2018-03-06
26     2018-04-21
27     2018-04-15
28     2018-03-16
29     2018-03-21
          ...    
121    2018-01-07
122    2018-01-13
123    2018-01-11
124    2018-01-17
125    2018-01-10
126    2018-01-08
127    2018-01-06
128    2018-01-22
129    2018-02-17
130    2018-01-26
131    2018-02-06
132    2018-02-09
133    2018-02-01
134    2018-02-18
135    2018-01-21
136    2018-02-24
137    2018-01-04
138    2018-01-19
139    2018-03-03
140    2018-02-10
141    2018-02-08
142    2018-01-02
143    2018-02-16
144    2018-02-14
145    201

In [65]:
le.fit(fechaUnicos['fecha'])
eventsWithLabel3['fecha'] = le.fit_transform(eventsWithLabel3['fecha'])

In [66]:
eventsWithLabel3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2319285 entries, 0 to 2319284
Data columns (total 29 columns):
timestamp            datetime64[ns]
event                int64
person               int64
url                  object
sku                  float64
model                int64
condition            int64
storage              int64
color                int64
skus                 object
search_term          int64
staticpage           object
campaign_source      int64
search_engine        int64
screen_resolution    int64
browser_version      int64
label                float64
eventos              object
dayweek              int64
fecha                int64
marca                int64
cantEventos          int64
city                 int64
region               int64
country              int64
device_type          int64
SO                   int64
channel              int64
new_vs_returning     object
dtypes: datetime64[ns](1), float64(2), int64(21), object(5)
memory usage: 530.8+ MB


In [76]:
eventsWithLabel3.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,fecha,marca,cantEventos,city,region,country,device_type,SO,channel,new_vs_returning
0,2018-05-18 00:11:59,9,10802,NaN,9288.0,103,2,3,28,NaN,...,137,6,9,414,90,7,1,7,2,New
1,2018-05-18 00:30:30,9,10802,NaN,9288.0,103,2,3,28,NaN,...,137,6,9,414,90,7,1,7,2,New
2,2018-05-18 00:07:23,6,10802,NaN,NaN,207,5,8,62,NaN,...,137,11,9,414,90,7,1,7,2,New
3,2018-05-18 00:11:56,2,10802,NaN,9288.0,103,2,3,28,NaN,...,137,6,9,414,90,7,1,7,2,New
4,2018-05-18 00:11:35,9,10802,NaN,9287.0,103,3,3,28,NaN,...,137,6,9,414,90,7,1,7,2,New


In [ ]:
testDF = eventsWithLabel3.loc[eventsWithLabel3['label'].isnull()]
testDF.head()

In [ ]:
trainingDF = eventsWithLabel3.loc[eventsWithLabel3['label'].notnull()]
trainingDF.head()

In [ ]:
trainingDF['storage'].value_counts()

In [ ]:
trainingDF['color'].value_counts()

In [ ]:
trainingDF['campaign_source'].value_counts()

In [ ]:
trainingDF['search_engine'].value_counts()

In [ ]:
trainingDF.info()

In [ ]:
label_array = np.array(trainingDF['label'])
data_array = np.array(trainingDF.drop(['label', 'static_page'], axis=1))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data_array, label_array, test_size = 0.30, random_state = 1)

In [ ]:
def logisticReg(x_train, x_test, y_train, y_test):
    logReg = LogisticRegression(solver='sag', random_state=1)
    logReg.fit(x_train, y_train)
    y_pred = logReg.predict(x_test)
    return np.sqrt(mean_squared_error(y_test, y_pred))
#TAMBIEN PUEDEN USAR PARA VER EL ACCURACY
#ACCURACY = accuracy_score(y_test, y_pred)

In [ ]:
def knn(k, x_train, x_test, y_train, y_test):
    knn = KNeighborsClassifier(n_neighbors=k, metric='manhattan')
    knn.fit(x_train, y_train)
    y_pred = knn.predict(x_test)
    return np.sqrt(mean_squared_error(y_test, y_pred))
#TAMBIEN PUEDEN USAR PARA VER EL ACCURACY
#ACCURACY = accuracy_score(y_test, y_pred)

In [ ]:
def decisionTree(max_depth, max_features, x_train, x_test, y_train, y_test):
    dt = DecisionTreeClassifier(max_depth=max_depth, max_features=max_features)
    dt.fit(x_train, y_train)
    y_pred = dt.predict(x_test)
    return np.sqrt(mean_squared_error(y_test, y_pred))
#TAMBIEN PUEDEN USAR PARA VER EL ACCURACY
#ACCURACY = accuracy_score(y_test, y_pred)

In [ ]:
def xgboost(max_depth, n_estimators, x_train, x_test, y_train, y_test):
    xgb = XGBClassifier(max_depth=max_depth, n_estimators=n_estimators)
    xgb.fit(x_train, y_train)
    y_pred = xgb.predict(x_test)
    return np.sqrt(mean_squared_error(y_test, y_pred))
#TAMBIEN PUEDEN USAR PARA VER EL ACCURACY
#ACCURACY = accuracy_score(y_test, y_pred)

In [ ]:
result = logisticReg(x_train, x_test, y_train, y_test)
result